In [ ]:
from citibike.citibike_utils import get_trip_duration_mins
from utils.datetime_utils import timestamp_to_date_col
from pyspark.sql.functions import create_map, lit

In [ ]:
# META-DATA
pipeline_id = dbutils.widgets.get("pipeline_id")
run_id = dbutils.widgets.get("run_id")
task_id = dbutils.widgets.get("task_id")
processed_timestamp = dbutils.widgets.get("processed_timestamp")
#ENVIRONMENT SPECIFIC
environment = dbutils.widgets.get("environment")
region = dbutils.widgets.get("region")
catalog_name = f"citibike_{environment}"
# TABLES_NAMES
src_table_name = f'{catalog_name}.01_bronze.jc_citibike'
tgt_table_name = f'{catalog_name}.02_silver.jc_citibike'

In [ ]:
df = spark.read.table(src_table_name)

In [ ]:

df = get_trip_duration_mins(spark, df, "started_at", "ended_at", "trip_duration_mins")
df = timestamp_to_date_col(df, "started_at", "trip_start_date")
df = df.withColumn('metadata', 
    create_map(
        lit('pipeline_id'), lit(pipeline_id),
        lit('run_id'), lit(run_id),
        lit('task_id'), lit(task_id),
        lit('processed_timestamp'), lit(processed_timestamp)
    )
)
df = df.select(
    "ride_id",
    "trip_start_date",
    "started_at",
    "ended_at",
    "start_station_name",
    "end_station_name",
    "trip_duration_mins",
    "metadata"
)

In [ ]:
(df.write
    .mode('overwrite')
    .option("overwriteSchema","true")
    .saveAsTable(tgt_table_name)
)